## Integrantes

- Jakelin Daiana Correa Palacio
- Andrés Felipe Lema García
- Luis Felipe Moreno Chamorro

## Librerías necesarias

In [1]:
# Notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# .py
from scipy.optimize import minimize
from scipy.optimize import differential_evolution, minimize, dual_annealing
from scipy.optimize import basinhopping, shgo
from matplotlib.animation import FuncAnimation
from matplotlib.colors import LogNorm
import matplotlib.animation as animation
from IPython.display import HTML, IFrame

import json
from itertools import permutations
import folium
from folium import plugins


## Parte 2: optimización combinatoria

Un vendedor debe hacer un recorrido por las siguientes ciudades de Colombia en su carro (no necesariamente en este orden):

Palmira
Pasto
Tuluá
Bogota
Pereira
Armenia
Manizales
Valledupar
Montería
Soledad
Cartagena
Barranquilla
Medellín
Bucaramanga
Cúcuta
Utilice colonias de hormigas y algoritmos genéticos para encontrar el orden óptimo. El costo de desplazamiento entre ciudades es la suma del valor de la hora del vendedor (es un parámetro que debe estudiarse), el costo de los peajes y el costo del combustible. Cada equipo debe definir en qué carro hace el recorrido el vendedor y de allí extraer el costo del combustible.

Adicionalmente represente con un gif animado o un video cómo se comporta la mejor solución usando un gráfico del recorrido en el mapa de Colombia

In [2]:
# Se leen los datos del json
with open("data.json",'r', encoding='utf-8') as file:
    datos = json.load(file)

# Lista de ciudades para indexar matrices
ciudades = datos["ciudades"]

# Matrices
matriz_distancia = datos["distancia"]
matriz_precio = datos["precio"]
matriz_tiempo = datos["tiempo"]

# Ubicaciones
ubicaciones = datos["ubicaciones"]

In [3]:
# Se leen los datos del json
with open("data.json",'r', encoding='utf-8') as file:
    datos = json.load(file)

# Lista de ciudades para indexar matrices
ciudades = datos["ciudades"]

# Matrices
matriz_distancia = datos["distancia"]
matriz_precio = datos["precio"]
matriz_tiempo = datos["tiempo"]

# Ubicaciones
ubicaciones = datos["ubicaciones"]

In [4]:
# Normalizacion de datos para el calculo de aptitud

max_dist = max(max(fila) for fila in matriz_distancia)
max_prec = max(max(fila) for fila in matriz_precio)
max_tiem = max(max(fila) for fila in matriz_tiempo)

In [5]:
# Devuelve los index de las ciudades
def indexCiudad(ciudad1, ciudad2):
    return ciudades.index(ciudad1), ciudades.index(ciudad2)

# Devuelve el valor de la matriz entre dos ciudades
def distancia(ciudad1, ciudad2):
    indice_ciudad1, indice_ciudad2= indexCiudad(ciudad1, ciudad2)
    return matriz_distancia[indice_ciudad1][indice_ciudad2]

def precio(ciudad1, ciudad2):
    indice_ciudad1, indice_ciudad2= indexCiudad(ciudad1, ciudad2)
    return matriz_precio[indice_ciudad1][indice_ciudad2]

def tiempo(ciudad1, ciudad2):
    indice_ciudad1, indice_ciudad2= indexCiudad(ciudad1, ciudad2)
    return matriz_tiempo[indice_ciudad1][indice_ciudad2]

# Genera una población inicial aleatoria de rutas
def generar_poblacion(num_individuos):
    poblacion = []
    for _ in range(num_individuos):
        ruta = np.random.permutation(ciudades)
        poblacion.append(list(ruta))
    return poblacion

# Evalúa la aptitud de una ruta (distancia, precio y tiempo)
def evaluar_aptitud(ruta):
    distancia_total = 0
    precio_total = 0
    tiempo_total = 0

    aptitud = 0
    for i in range(len(ruta)):
        ciudad_actual = ruta[i]
        ciudad_siguiente = ruta[(i + 1) % len(ruta)]
        distancia_total += distancia(ciudad_actual, ciudad_siguiente)
        precio_total += precio(ciudad_actual, ciudad_siguiente)
        tiempo_total += tiempo(ciudad_actual, ciudad_siguiente)

        costo = distancia(ciudad_actual, ciudad_siguiente)/ max_dist + precio(ciudad_actual, ciudad_siguiente)/ max_prec + tiempo(ciudad_actual, ciudad_siguiente)/ max_tiem
        aptitud += costo
        
    return aptitud, distancia_total, precio_total, tiempo_total

# Cruza dos rutas para producir descendencia
def cruzamiento(padre1, padre2):
    punto_corte = np.random.randint(1, len(padre1) - 1)
    hijo1 = padre1[:punto_corte] + [ciudad for ciudad in padre2 if ciudad not in padre1[:punto_corte]]
    hijo2 = padre2[:punto_corte] + [ciudad for ciudad in padre1 if ciudad not in padre2[:punto_corte]]
    return hijo1, hijo2

# Aplica mutaciones a la descendencia
def mutacion(ruta, prob_mutacion):
    if np.random.rand() < prob_mutacion:
        idx1, idx2 = np.random.choice(len(ruta), size=2, replace=False)
        ruta[idx1], ruta[idx2] = ruta[idx2], ruta[idx1]
    return ruta

# Algoritmo genético para resolver el problema
def algoritmo_genetico(num_generaciones, tam_poblacion, prob_mutacion):
    poblacion = generar_poblacion(tam_poblacion)
    mejor_aptitud = float('inf')
    mejor_ruta = None
    for generacion in range(num_generaciones):
        descendientes = []
        for _ in range(tam_poblacion // 2):
            padres = np.random.choice(len(poblacion), size=2, replace=False)
            hijo1, hijo2 = cruzamiento(poblacion[padres[0]], poblacion[padres[1]])
            hijo1 = mutacion(hijo1, prob_mutacion)
            hijo2 = mutacion(hijo2, prob_mutacion)
            descendientes.extend([hijo1, hijo2])
        poblacion = descendientes
        for individuo in poblacion:
            aptitud, _, _, _ = evaluar_aptitud(individuo)
            if aptitud < mejor_aptitud:
                mejor_aptitud = aptitud
                mejor_ruta = individuo
    return mejor_ruta

In [6]:
def mostrarAnimacionMapa(ruta):
    # Se obtiene la ubicacion de cada ciudad
    latitudes, longitudes = [], []
    
    for ciudad in ruta:
        latitudes.append(ubicaciones[ciudad][0])
        longitudes.append(ubicaciones[ciudad][1])
    
    # Se crea el mapa de folium
    mapa = folium.Map(location=[sum(latitudes)/len(latitudes), sum(longitudes)/len(longitudes)], zoom_start=6)
    
    # Se crea la ruta como un PolyLine al mapa
    ruta = folium.PolyLine(locations=list(zip(latitudes, longitudes)), color='blue', weight=3)
    mapa.add_child(ruta)
    
    # Crear un plugin de animación para la ruta
    animacion_ruta = plugins.AntPath(locations=list(zip(latitudes, longitudes)), dash_array=[10, 20], delay=1000)
    mapa.add_child(animacion_ruta)
    
    # Guardar el mapa como un archivo HTML
    mapa.save('animacion_ruta.html')

In [7]:
# Parámetros del algoritmo genético
num_generaciones = 100
tam_poblacion = 100
prob_mutacion = 0.02

# Ejecucion del algoritmo genético
ruta_optima = algoritmo_genetico(num_generaciones, tam_poblacion, prob_mutacion=0.02)
print("Ruta optima encontrada:", ruta_optima)
print("Aptitud: {:.2f}\nDistancia: {:.3f}km\nPrecio: ${}\nTiempo total: {} minutos".format(*evaluar_aptitud(ruta_optima)))

# se genera un archivo .html para mostrar la animacion
mostrarAnimacionMapa(ruta_optima)
# este archivo es llamado desde un iFrame
IFrame(src='animacion_ruta.html', width=800, height=600)

Ruta optima encontrada: ['Tuluá', 'Manizales', 'Bucaramanga', 'Cúcuta', 'Montería', 'Pasto', 'Pereira', 'Armenia', 'Medellín', 'Valledupar', 'Cartagena', 'Soledad', 'Barranquilla', 'Bogotá', 'Palmira']
Aptitud: 11.80
Distancia: 4248.700km
Precio: $2660749
Tiempo total: 7328 minutos


In [8]:
# Se ejecuta 50 veces el algoritmo hasta encontrar la mejor
all_routes = []
aptitud_min = 0
best_ruta = {}

for i in range(50):
    ejecucion = {"ejecucion" : i}
    
    # Ejecucion del algoritmo genético
    ruta_optima = algoritmo_genetico(num_generaciones, tam_poblacion, prob_mutacion=0.02)
    ejecucion["ruta"] = ruta_optima
    
    ejecucion["aptitud"], ejecucion["distancia"], ejecucion["precio"], ejecucion["tiempo"] = evaluar_aptitud(ruta_optima)
    
    # si es la primera ejecucion o la aptitud es menor a la que se habia encontrado...
    if i == 0 or best_ruta["aptitud"] > ejecucion["aptitud"]:
        best_ruta = ejecucion
        
    all_routes.append(ejecucion)

# Se muestran por consola los atributos de la mejor ruta
print("Ruta optima encontrada:", best_ruta["ruta"])
print("Aptitud:", round(ejecucion["aptitud"], 2))
print("Distancia:", round(ejecucion["distancia"], 3), "km")
print("Precio: $", ejecucion["precio"])
print("Tiempo total:", ejecucion["tiempo"], "minutos")

# Visualizar la mejor ruta
mostrarAnimacionMapa(best_ruta["ruta"])
IFrame(src='animacion_ruta.html', width=800, height=600)

Ruta optima encontrada: ['Bucaramanga', 'Cúcuta', 'Valledupar', 'Tuluá', 'Pereira', 'Armenia', 'Palmira', 'Pasto', 'Montería', 'Soledad', 'Cartagena', 'Barranquilla', 'Bogotá', 'Manizales', 'Medellín']
Aptitud: 13.32
Distancia: 5894.4 km
Precio: $ 2968957
Tiempo total: 6966 minutos
